In [28]:
# Импортируем Библиотеки

import numpy as np
import cv2
from scipy.interpolate import UnivariateSpline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage import measure
from scipy import ndimage 
import tkinter as tk
from tkinter import filedialog
%matplotlib inline

Цель: 
        В данном проекте мы применяем различные фильтры к фотографии и сохроняем ее с фильтром

Что получлось:
        Получилось освоить основные инструменты корректировки цветов и корректировки резкости.
        Получилось применять фильтры на фото, предварительно просматривать их и сохранять.

Что не получилось:
        Не получилось создать кнопку "Сохранить" для сохраниня на финальной стадии картинки с фильтром.
        Хотелось бы после применения фильтра получать переменную, которая хранила бы информацию о файле
        и примененном фильтре, далее код бы предлагал куда сохранить новый файл.
    





In [45]:
root=tk.Tk() #Вызываем диалоговое окно и выбираем файл для применения цветового фильтра
root.withdraw()
file_path=filedialog.askopenfilename()

In [46]:
path=cv2.imread(file_path)
img=cv2.cvtColor(path, cv2.COLOR_BGR2RGB)
plt.rcParams['figure.figsize']=[15,15]

def mapping_function(x,y):
    spl=UnivariateSpline(x,y)
    return spl(range(256))

In [31]:
def apply_warm(image):#Теплый оттенок
    increase=mapping_function([0,64,128,192,256],[0,70,140,210,256])
    decrase=mapping_function([0,64,128,192,256],[0,40,90,150,256])
    red,green,blue=cv2.split(image)
    red=cv2.LUT(red,increase).astype(np.uint8)
    blue=cv2.LUT(blue,decrase).astype(np.uint8)   
    imige=cv2.merge((red,green,blue))
    return imige

In [32]:
def apply_cool(image): #Холодный оттенок
    increase=mapping_function([0,64,128,192,256],[0,70,140,210,256])
    decrase=mapping_function([0,64,128,192,256],[0,40,90,150,256])
    red,green,blue=cv2.split(image)
    red=cv2.LUT(red,decrase).astype(np.uint8)
    blue=cv2.LUT(blue,increase).astype(np.uint8)   
    imige=cv2.merge((red,green,blue))
    return imige

In [33]:
def Rotated_90(img): # Поворот на 90 градусов
    (h, w, d) = img.shape
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, 90, 1.0)   
    rotated = cv2.warpAffine(img, M, (w, h))  
    return rotated
def Rotated_180(img):# Поворот на 180 градусов
    (h, w, d) = img.shape
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, 180, 1.0)   
    rotated = cv2.warpAffine(img, M, (w, h))  
    return rotated

In [34]:
def increase_brightness(img, value=10): # Регулировка яркости
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value
    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [35]:
def Sharpening(img):    #Увеличение резкости
    kernel = np.array([[-1,-1,-1], 
                       [-1, 9,-1],
                       [-1,-1,-1]])
    sharpened = cv2.filter2D(path,-1,kernel)
    #rgb_image = cv2.cvtColor(sharpened, cv2.COLOR_BGR2RGB) - в данной фунции не получается реализовать, т.к неккоректно выводится цветовой фон
    return  sharpened #rgb_image 


In [36]:
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
# Прописываем условия для вывода:

def choice (x, img):
    if x=='Warm':
        return plt.imshow(apply_warm(img))
       # new_img=apply_warm(img)- надеялся, что получится таким образом создать переменную для дальнейшего сохранения
    if x == 'Rotated 90':
        return plt.imshow(Rotated_90(img))
       # new_img=Rotated_90(img)
    if x == 'Rotated 180':
        return plt.imshow(Rotated_180(img))
       # new_img=Rotated_180(img)
    if x =='Cool':
        return plt.imshow(apply_cool(img))
       # new_img=apply_cool(img)
    
    if x =='increase_brightness':
        return plt.imshow(increase_brightness(img))
       # new_img=increase_brightness(img)
    if x =='Sharpening':                    
        return plt.imshow(Sharpening(img))
      #  new_img=Sharpening(img)
    if x == 'No filter':
        return plt.imshow(img)

In [47]:
interact(choice, x=widgets.Dropdown(options=['No filter','Cool',
                                             'Warm','Sharpening',
                                             'increase_brightness',
                                             'Rotated 90',
                                             'Rotated 180'],description='Filter'), img=fixed(img))
#Фильтр Shaping отображается в блокноте не корректно, но при выводе, файл сохраняется отлично


interactive(children=(Dropdown(description='Filter', options=('No filter', 'Cool', 'Warm', 'Sharpening', 'incr…

<function __main__.choice(x, img)>

In [48]:
img_BGR=cv2.cvtColor(img, cv2.COLOR_RGB2BGR) #переводим в канал RGB для корреткного вывода файла
# далеее вводим смостоятельно наименование файла и выбираем функцию
img_filtr=apply_cool(img) #Заменяем apply_cool на любой понравившeйся фильтр :
                          # apply_warm-из-за конвертации цветов добавит холодный оттенок
                          # apply_cool-из-за конвертации цветов добавит теплый оттенок
                          # Rotated_90-Поворот на 90 градусов влево
                          # Rotated_180-Поворот на 180 градусов
                          # increase_brightness - добавить яркость
                          # Sharpening- добавить резкость                                                                            """                                                                      
                                                                                                                                                      
                                                                            
cv2.imwrite('test2.jpg', img_filtr) #Сохраняем файл (хотелось бы добавить диалоговое окно с выбором места для сохранения)

True